In [36]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config('spark.shuffle.useOldFetchProtocol','true'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [37]:
spark

In [38]:
raw_df = spark.read.format("csv").option("inferSchema", True).option("header",True).load("/user/itv017244/lendingclub/accepted_2007_to_2018Q4.csv")

In [39]:
raw_df.createOrReplaceTempView("lending_club_date")

In [40]:
spark.sql("select * from lending_club_date")

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
68407277,null,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingcl...,null,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,null,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723916800001,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,null,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,null,null,null,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,null,null,null,null,null,null,null,null,null,null,null,null,null,N,null,null,null,null,null,null,null,null,null,null,null,null,null,null,Cash,N,null,null,null,null,null,null
68355089,null,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingcl...,null,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,null,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.66,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,null,Mar-2019,699.0,695.0,0.0,null,1.0,Individual,null,null,null,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,null,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,null,null,null,null,null,null,null,null

In [41]:
from pyspark.sql.functions import sha2, concat_ws

In [42]:
new_df = raw_df.withColumn("name_sha2",sha2(concat_ws("||",*["emp_title","emp_length","home_ownership","annual_inc","zip_code","addr_state","grade","sub_grade","verification_status"]),256))

In [43]:
new_df.createOrReplaceTempView("new_Table")

In [44]:
spark.sql("select count(*) from new_Table")

count(1)
2260701


In [45]:
spark.sql("select count(distinct(name_sha2)) from new_Table")

count(DISTINCT name_sha2)
2257384


In [46]:
spark.sql("select name_sha2,count(*) as total_cnt from new_Table group by name_sha2 having total_cnt>1 order by total_cnt desc")

name_sha2,total_cnt
e3b0c44298fc1c149...,33
e4c167053d5418230...,5
3f87585a20f702838...,4
76b577467eda5bdbc...,4
ad8e5d384dae17e06...,4
f54295a60946dedad...,3
53789bea7edc660ed...,3
bf3d06b33c49a9375...,3
059d401bb603d9a80...,3
819453be77718d747...,3


In [21]:
spark.sql("select * from new_table where name_sha2 like 'e3b0c44%'")

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,name_sha2
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c44298fc1c149...
Total amount fund...,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,e3b0c

In [23]:
spark.sql("""select name_sha2 as member_id, emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,'USA' as country,
grade, sub_grade, verification_status, tot_hi_cred_lim, application_type, annual_inc_joint,verification_status_joint from new_Table
""").repartition(1).write.option("header",True).format("csv").mode("overwrite"). \
option("path","/user/itv017244/lendingclubproject/raw/customer_data_csv").save()

In [24]:
customer_df = spark.read.format("csv").option("inferSchema", True).option("header",True).load("/user/itv017244/lendingclubproject/raw/customer_data_csv")

In [25]:
customer_df

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
d86a1c7244ace602c...,Admin,3 years,RENT,31000.0,OR,970xx,USA,B,B3,Source Verified,49481.0,Individual,null,null
25b8cd9ff47e50eea...,Teacher,10+ years,MORTGAGE,55983.0,TX,780xx,USA,C,C2,Source Verified,210551.0,Individual,null,null
f9773942305b2dc9c...,Behavioral Asst,10+ years,MORTGAGE,40000.0,VA,201xx,USA,F,F1,Source Verified,272608.0,Individual,null,null
b22f99c198f3738a7...,owner,< 1 year,RENT,20000.0,NC,274xx,USA,C,C3,Source Verified,58521.0,Individual,null,null
4b3b7068c15b0e046...,Supervisor,< 1 year,RENT,45000.0,NY,117xx,USA,C,C5,Source Verified,25500.0,Individual,null,null
0c7a9f8a6d5b209d2...,case manager,1 year,RENT,35000.0,NV,891xx,USA,B,B1,Verified,37588.0,Individual,null,null
f1cc7e6ae9c735148...,null,null,MORTGAGE,90000.0,AZ,851xx,USA,B,B5,Verified,303921.0,Individual,null,null
8f8faa7eeb2a1ee41...,Merchandiser,6 years,RENT,22000.0,NY,145xx,USA,D,D1,Source Verified,17650.0,Individual,null,null
dad3f5ea8788186cb...,executive assistant,1 year,RENT,41000.0,WA,981xx,USA,C,C5,Verified,29975.0,Individual,null,null
a20867c0985126202...,Industrial Tech,10+ years,MORTGAGE,75000.0,KS,676xx,USA,B,B4,Not Verified,68593.0,Individual,null,null


In [26]:
spark.sql("""select id as loan_id,name_sha2 as member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d
loan_status,purpose,title from new_Table
""").repartition(1).write.option("header",True).format("csv").mode("overwrite"). \
option("path","/user/itv017244/lendingclubproject/raw/loans_data_csv").save()

In [27]:
loans_df = spark.read.format("csv").option("inferSchema", True).option("header",True).load("/user/itv017244/lendingclubproject/raw/loans_data_csv")

In [28]:
loans_df

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,loan_status,purpose,title
14408468,5d7676571dee53d06...,5000.0,5000.0,36 months,14.16,171.28,Apr-2014,debt_consolidation,Debt consolidation
14520567,31fd1ef036c5caf26...,11000.0,11000.0,36 months,9.67,353.24,Apr-2014,other,Other
14708730,9b48253ca5848fa78...,25000.0,25000.0,36 months,11.99,830.24,May-2014,credit_card,Credit card refin...
14491003,d6b8f2e32be148721...,1500.0,1500.0,36 months,23.43,58.41,Apr-2014,renewable_energy,Green loan
14510981,0a81ad556e20bcdb9...,10000.0,10000.0,36 months,12.99,336.9,Apr-2014,debt_consolidation,Debt consolidation
14177845,843b995d39160ddab...,13200.0,13200.0,60 months,12.99,300.28,Apr-2014,debt_consolidation,Debt consolidation
13947687,04a8bd7c4e1407ba8...,10000.0,10000.0,60 months,17.57,251.61,Apr-2014,other,Other
12905731,3e9a89fa96988af68...,18000.0,18000.0,36 months,18.25,653.01,May-2014,home_improvement,Home improvement
14137736,394a8db0bd3bcf7de...,12000.0,12000.0,36 months,9.67,385.35,Apr-2014,debt_consolidation,Debt consolidation
13207083,09cea8d2b6f1272f7...,3200.0,3200.0,36 months,20.49,119.73,Apr-2014,credit_card,Credit card refin...


In [29]:
spark.sql("""select id as loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,
total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d  from new_Table
""").repartition(1).write.option("header",True).format("csv").mode("overwrite"). \
option("path","/user/itv017244/lendingclubproject/raw/loans_repayments_csv").save()

In [30]:
loans_repayments_df = spark.read.format("csv").option("inferSchema", True).option("header",True).load("/user/itv017244/lendingclubproject/raw/loans_repayments_csv")

In [31]:
loans_repayments_df

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
56633077,3000.0,376.21,0.0,3376.205975527,93.74,Aug-2018,null
55927518,15600.0,1956.32,0.0,17556.320693408998,487.9,Aug-2018,null
56473345,20000.0,2408.94,0.0,22408.9398096902,9677.72,May-2017,null
56463188,11200.0,5231.01,0.0,16431.0146429476,7475.86,Feb-2018,null
56473316,5215.47,6513.51,0.0,13237.07,432.77,Nov-2017,null
56663266,6477.26,4221.77,0.0,12544.05,458.45,Jun-2017,null
56483027,10000.0,2062.03,0.0,12062.026276051001,335.38,Aug-2018,null
56613385,13932.74,11678.23,0.0,25610.97,609.46,Feb-2019,Apr-2019
56643620,16000.0,1031.67,0.0,17031.673055266598,8363.28,Mar-2017,null
56533114,3478.66,3815.07,0.0,12460.86,1111.52,Mar-2016,null


In [47]:
spark.sql("""select name_sha2 as member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,
total_rec_late_fee, mths_since_last_delinq,mths_since_last_record from new_Table
""").repartition(1).write.option("header",True).format("csv").mode("overwrite"). \
option("path","/user/itv017244/lendingclubproject/raw/loans_defaulters_csv").save()

In [49]:
loans_defaulters_df = spark.read.format("csv").option("inferSchema", True).option("header",True).load("/user/itv017244/lendingclubproject/raw/loans_defaulters_csv")

In [50]:
loans_defaulters_df

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
bfcec8da7b3f1fe83...,1.0,0.0,0.0,0.0,2.0,0.0,11.0,null
36399458db4295868...,0.0,0.0,1.0,1.0,0.0,0.0,null,63.0
e4912b650878ca941...,0.0,0.0,0.0,0.0,0.0,0.0,67.0,null
3d6f35ad2e2be4572...,0.0,0.0,1.0,1.0,0.0,0.0,null,66.0
478e1cd37e35c20f7...,1.0,0.0,1.0,1.0,0.0,0.0,15.0,108.0
a5d42e35cc0be2789...,0.0,0.0,1.0,1.0,1.0,0.0,null,83.0
7f3ef55c784bf6b16...,0.0,0.0,0.0,0.0,0.0,0.0,41.0,null
beb30abaeca08f49b...,2.0,0.0,0.0,0.0,0.0,0.0,14.0,null
233e5e6797dd0a68b...,0.0,0.0,1.0,1.0,1.0,0.0,58.0,85.0
c4b178ffaf80ed472...,1.0,0.0,0.0,0.0,0.0,0.0,17.0,null
